<a href="https://colab.research.google.com/github/vinee-sha/Social-Distancing-Tracker/blob/main/TrainHuman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [120]:
!unzip "/content/gdrive/MyDrive/HumanDataset.zip" -d "/content"

Archive:  /content/gdrive/MyDrive/HumanDataset.zip
   creating: /content/Human Dataset/Train/Human/
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_078.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_079.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_080.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_081.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_082.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_083.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_085.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_086.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_087.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_088.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_175.jpg  
  inflating: /content/Human Dataset/Train/Human/brushing_teeth_176.jpg  
  inflating: /content/Hu

In [121]:
train_dir = "/content/HumanDataset/Train"
validation_dir = "/content/HumanDataset/Validation"

In [81]:
# train_humans_dir = os.path.join(train_dir, 'Train')
# validation_humans_dir = os.path.join(validation_dir, 'Validation')

In [122]:
total_train = len(os.listdir(train_dir)) 
total_val = len(os.listdir(validation_dir)) 
print(total_train, total_val)

FileNotFoundError: ignored

In [82]:
# Model Parameters
BATCH_SIZE = 120
IMG_SHAPE = 150
EPOCHS = 100

MODEL CREATION

In [83]:
# Initialize the CNN
classifier = Sequential()

In [84]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape=(IMG_SHAPE, IMG_SHAPE, 3), activation='relu'))

In [85]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(2, 2))

In [86]:
# Step 1 & 2 with more hidden layers
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

classifier.add(Conv2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(2, 2))

In [87]:
# Step 3 - Dropout
classifier.add(Dropout(0.5))

In [88]:
# Step 4 - Flattening
classifier.add(Flatten())

In [89]:
# Step 5 - Full connection
classifier.add(Dense(units=512, activation='relu'))

In [93]:
# Step 6 - Output layer
classifier.add(Dense(units=1, activation='softmax'))

In [94]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [95]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

Data Augmentation and Preparation

In [117]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
) 

In [118]:
training_set = train_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory="/content/Human Dataset",
    shuffle=True,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='binary'
)

Found 1506 images belonging to 2 classes.


In [119]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [99]:
validation_set = validation_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=validation_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='binary'
)

Found 0 images belonging to 0 classes.


In [78]:
history = classifier.fit_generator(
    training_set,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=validation_set,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: ignored